# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 14 2022 11:39AM,246410,10,BPS200084,Beach Patient Assistance,Released
1,Sep 14 2022 11:39AM,246410,10,8525536,Beach Products Inc,Released
2,Sep 14 2022 11:39AM,246410,10,8525537,Beach Products Inc,Released
3,Sep 14 2022 11:39AM,246410,10,8525538,Beach Products Inc,Released
4,Sep 14 2022 11:39AM,246410,10,8525539,Beach Products Inc,Released
5,Sep 14 2022 11:39AM,246410,10,8525542,Beach Products Inc,Released
6,Sep 14 2022 11:39AM,246410,10,8525543,Beach Products Inc,Released
7,Sep 14 2022 11:39AM,246410,10,8525544,Beach Products Inc,Released
8,Sep 14 2022 11:38AM,246408,10,Enova-10637,Emerginnova,Released
9,Sep 14 2022 11:36AM,246407,10,EYEFBA16W4H01F4U000001,Eye Pharma Inc,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
43,246404,Released,1
44,246406,Released,2
45,246407,Released,1
46,246408,Released,1
47,246410,Released,8


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
246404,NaN,NaN,1.0
246406,NaN,NaN,2.0
246407,NaN,NaN,1.0
246408,NaN,NaN,1.0
246410,NaN,NaN,8.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
246185,0.0,1.0,0.0
246199,0.0,1.0,0.0
246201,0.0,0.0,1.0
246202,0.0,0.0,1.0
246203,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
246185,0,1,0
246199,0,1,0
246201,0,0,1
246202,0,0,1
246203,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,246185,0,1,0
1,246199,0,1,0
2,246201,0,0,1
3,246202,0,0,1
4,246203,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,246185,,1,
1,246199,,1,
2,246201,,,1
3,246202,,,1
4,246203,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 14 2022 11:39AM,246410,10,Beach Patient Assistance
1,Sep 14 2022 11:39AM,246410,10,Beach Products Inc
8,Sep 14 2022 11:38AM,246408,10,Emerginnova
9,Sep 14 2022 11:36AM,246407,10,Eye Pharma Inc
10,Sep 14 2022 11:35AM,246392,10,"Senseonics, Incorporated"
15,Sep 14 2022 11:35AM,246406,19,"GCH Granules USA, Inc."
17,Sep 14 2022 11:23AM,246404,10,"Senseonics, Incorporated"
18,Sep 14 2022 11:22AM,246403,19,"Methapharm, Inc."
22,Sep 14 2022 11:15AM,246402,19,ACG North America LLC
26,Sep 14 2022 10:57AM,246399,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Sep 14 2022 11:39AM,246410,10,Beach Patient Assistance,,,8
1,Sep 14 2022 11:39AM,246410,10,Beach Products Inc,,,8
2,Sep 14 2022 11:38AM,246408,10,Emerginnova,,,1
3,Sep 14 2022 11:36AM,246407,10,Eye Pharma Inc,,,1
4,Sep 14 2022 11:35AM,246392,10,"Senseonics, Incorporated",,,5
5,Sep 14 2022 11:35AM,246406,19,"GCH Granules USA, Inc.",,,2
6,Sep 14 2022 11:23AM,246404,10,"Senseonics, Incorporated",,,1
7,Sep 14 2022 11:22AM,246403,19,"Methapharm, Inc.",,,4
8,Sep 14 2022 11:15AM,246402,19,ACG North America LLC,,,4
9,Sep 14 2022 10:57AM,246399,10,ISDIN Corporation,,,14


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 14 2022 11:39AM,246410,10,Beach Patient Assistance,8,,
1,Sep 14 2022 11:39AM,246410,10,Beach Products Inc,8,,
2,Sep 14 2022 11:38AM,246408,10,Emerginnova,1,,
3,Sep 14 2022 11:36AM,246407,10,Eye Pharma Inc,1,,
4,Sep 14 2022 11:35AM,246392,10,"Senseonics, Incorporated",5,,
5,Sep 14 2022 11:35AM,246406,19,"GCH Granules USA, Inc.",2,,
6,Sep 14 2022 11:23AM,246404,10,"Senseonics, Incorporated",1,,
7,Sep 14 2022 11:22AM,246403,19,"Methapharm, Inc.",4,,
8,Sep 14 2022 11:15AM,246402,19,ACG North America LLC,4,,
9,Sep 14 2022 10:57AM,246399,10,ISDIN Corporation,14,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 14 2022 11:39AM,246410,10,Beach Patient Assistance,8,,
1,Sep 14 2022 11:39AM,246410,10,Beach Products Inc,8,,
2,Sep 14 2022 11:38AM,246408,10,Emerginnova,1,,
3,Sep 14 2022 11:36AM,246407,10,Eye Pharma Inc,1,,
4,Sep 14 2022 11:35AM,246392,10,"Senseonics, Incorporated",5,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 14 2022 11:39AM,246410,10,Beach Patient Assistance,8.0,NaN,NaN
1,Sep 14 2022 11:39AM,246410,10,Beach Products Inc,8.0,NaN,NaN
2,Sep 14 2022 11:38AM,246408,10,Emerginnova,1.0,NaN,NaN
3,Sep 14 2022 11:36AM,246407,10,Eye Pharma Inc,1.0,NaN,NaN
4,Sep 14 2022 11:35AM,246392,10,"Senseonics, Incorporated",5.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 14 2022 11:39AM,246410,10,Beach Patient Assistance,8.0,0.0,0.0
1,Sep 14 2022 11:39AM,246410,10,Beach Products Inc,8.0,0.0,0.0
2,Sep 14 2022 11:38AM,246408,10,Emerginnova,1.0,0.0,0.0
3,Sep 14 2022 11:36AM,246407,10,Eye Pharma Inc,1.0,0.0,0.0
4,Sep 14 2022 11:35AM,246392,10,"Senseonics, Incorporated",5.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3942050,54.0,10.0,0.0
102,985422,4.0,0.0,0.0
12,246330,0.0,1.0,0.0
15,246384,11.0,12.0,14.0
16,246374,1.0,0.0,0.0
17,739102,3.0,0.0,0.0
19,1231992,12.0,0.0,0.0
20,985351,3.0,7.0,0.0
21,2216016,7.0,2.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3942050,54.0,10.0,0.0
1,102,985422,4.0,0.0,0.0
2,12,246330,0.0,1.0,0.0
3,15,246384,11.0,12.0,14.0
4,16,246374,1.0,0.0,0.0
5,17,739102,3.0,0.0,0.0
6,19,1231992,12.0,0.0,0.0
7,20,985351,3.0,7.0,0.0
8,21,2216016,7.0,2.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,54.0,10.0,0.0
1,102,4.0,0.0,0.0
2,12,0.0,1.0,0.0
3,15,11.0,12.0,14.0
4,16,1.0,0.0,0.0
5,17,3.0,0.0,0.0
6,19,12.0,0.0,0.0
7,20,3.0,7.0,0.0
8,21,7.0,2.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,54.0
1,102,Released,4.0
2,12,Released,0.0
3,15,Released,11.0
4,16,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,102,12,15,16,17,19,20,21
Status,,,,,,,,,
Completed,0.0,0.0,0.0,14.0,0.0,0.0,0.0,0.0,0.0
Executing,10.0,0.0,1.0,12.0,0.0,0.0,0.0,7.0,2.0
Released,54.0,4.0,0.0,11.0,1.0,3.0,12.0,3.0,7.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,102,12,15,16,17,19,20,21
0,Completed,0.0,0.0,0.0,14.0,0.0,0.0,0.0,0.0,0.0
1,Executing,10.0,0.0,1.0,12.0,0.0,0.0,0.0,7.0,2.0
2,Released,54.0,4.0,0.0,11.0,1.0,3.0,12.0,3.0,7.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,102,12,15,16,17,19,20,21
0,Completed,0.0,0.0,0.0,14.0,0.0,0.0,0.0,0.0,0.0
1,Executing,10.0,0.0,1.0,12.0,0.0,0.0,0.0,7.0,2.0
2,Released,54.0,4.0,0.0,11.0,1.0,3.0,12.0,3.0,7.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()